In [1]:
import torch
from sklearn.metrics import classification_report
%run model.ipynb

In [2]:
checkpoint = torch.load('checkpoint/incart/checkpoint_best.pt')

/tmp/ipykernel_1679851/2750279563.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('checkpoint/incart/checkpoint_best.pt')


In [3]:
checkpoint

{'dataset': 'incart',
 'configs': {'num_classes': 2,
  'class_names': ['normal', 'abnormal'],
  'sequence_len': 186,
  'input_channels': 1,
  'kernel_size': 8,
  'stride': 1,
  'dropout': 0.2,
  'mid_channels': 32,
  'final_out_channels': 128,
  'trans_dim': 25,
  'num_heads': 5},
 'hparams': {'num_epochs': 60,
  'batch_size': 128,
  'weight_decay': 0.0001,
  'learning_rate': 0.001,
  'feature_dim': 128},
 'model': OrderedDict([('conv4.weight',
               tensor([[[-1.0670e-01,  1.5296e-01, -4.3962e-01]],
               
                       [[-9.3737e-24, -4.1652e-24,  7.7881e-23]],
               
                       [[ 6.3745e-02,  4.9598e-01,  1.1464e-01]],
               
                       [[ 1.8833e-01, -1.9971e-01, -2.0553e-01]],
               
                       [[-6.1116e-01, -3.8724e-01, -1.4323e-01]],
               
                       [[ 7.2781e-03,  2.9845e-01,  4.3315e-01]],
               
                       [[-4.2223e-01, -2.0902e-01,  2.6993e

In [4]:
configs = checkpoint['configs']
hparams = checkpoint['hparams']

In [5]:
hparams

{'num_epochs': 60,
 'batch_size': 128,
 'weight_decay': 0.0001,
 'learning_rate': 0.001,
 'feature_dim': 128}

In [6]:
class Configs:
    def __init__(self, **entries):
        self.__dict__.update(entries)

    def __getattr__(self, item):
        return getattr(self.__dict__, item)

class HParams:
    def __init__(self, **entries):
        self.__dict__.update(entries)

    def __getitem__(self, key):
        return getattr(self, key)


# 创建Configs和HParams对象
configs = Configs(**checkpoint['configs'])
hparams = HParams(**checkpoint['hparams'])

In [7]:
model = Model(configs,hparams)


/home/qziedu/anaconda3/lib/python3.12/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [8]:
checkpoint['model']

OrderedDict([('conv4.weight',
              tensor([[[-1.0670e-01,  1.5296e-01, -4.3962e-01]],
              
                      [[-9.3737e-24, -4.1652e-24,  7.7881e-23]],
              
                      [[ 6.3745e-02,  4.9598e-01,  1.1464e-01]],
              
                      [[ 1.8833e-01, -1.9971e-01, -2.0553e-01]],
              
                      [[-6.1116e-01, -3.8724e-01, -1.4323e-01]],
              
                      [[ 7.2781e-03,  2.9845e-01,  4.3315e-01]],
              
                      [[-4.2223e-01, -2.0902e-01,  2.6993e-01]],
              
                      [[-2.0903e-02, -1.3797e-02, -1.7697e-02]],
              
                      [[ 9.5573e-14, -1.6803e-13,  3.6072e-14]],
              
                      [[-4.5728e-01, -3.7267e-01, -2.2147e-01]],
              
                      [[ 1.2153e-02,  2.7212e-02,  3.0292e-02]],
              
                      [[-2.4189e-01, -3.5215e-01, -3.8588e-01]],
              
          

In [9]:
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [10]:
model.eval()

test_loader = torch.load(f"/home/qziedu/test_hsj/main/data/incart/testinno.pt")

/tmp/ipykernel_1679851/2394429956.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test_loader = torch.load(f"/home/qziedu/test_hsj/main/data/incart/testinno.pt")


In [11]:
test_loader

{'samples': tensor([[[-0.0240, -0.0246, -0.0241,  ..., -0.0728, -0.0768, -0.0754]],
 
         [[-0.9528, -0.9340, -0.9473,  ..., -1.0279, -1.0335, -0.9378]],
 
         [[-0.4290, -0.4307, -0.4384,  ..., -0.4737, -0.4783, -0.4801]],
 
         ...,
 
         [[ 0.0888,  0.0855,  0.0846,  ...,  0.1510,  0.1560,  0.1577]],
 
         [[-2.7190, -2.6794, -2.6371,  ..., -2.5338, -2.5300, -2.5252]],
 
         [[-1.0327, -1.0286, -1.0249,  ..., -1.0689, -1.0679, -1.0683]]],
        dtype=torch.float64),
 'labels': tensor([0, 1, 0,  ..., 1, 1, 1])}

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 将模型移动到设备上
model.to(device)

# 从字典中提取数据和标签
samples = test_loader['samples'].to(device=device, dtype=torch.float32)  # 转换为 Float 类型并移动到设备
labels = test_loader['labels'].to(device=device,dtype=torch.long)  # 标签通常需要是 Long 类型并移动到设备

# 进行预测
all_predictions = []
all_labels = []
batch_size=128
with torch.no_grad():
    for i in range(0, len(samples), batch_size):
        batch_samples = samples[i:i+batch_size].to(device=device, dtype=torch.float32)
        batch_labels = labels[i:i+batch_size].to(device=device, dtype=torch.long)
        
        outputs = model(batch_samples)
        max_values, predicted = torch.max(outputs, 1)
        
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(batch_labels.cpu().numpy())

# 计算准确率
accuracy = sum(p == t for p, t in zip(all_predictions, all_labels)) / len(all_labels)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 99.55%


In [13]:
# 生成分类报告
report = classification_report(all_labels, all_predictions, target_names=None,digits=4) 
print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

           0     0.9969    0.9980    0.9974     30735
           1     0.9859    0.9786    0.9823      4440

    accuracy                         0.9955     35175
   macro avg     0.9914    0.9883    0.9899     35175
weighted avg     0.9955    0.9955    0.9955     35175

